In [21]:
import psycopg2
import time
import csv

# SERVER SIDE CURSOR FOR REDDIT POST SENTIMENT ANALYSIS


from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


#engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))

# connect:
con = None
con = psycopg2.connect(database = 'christle', user = 'christle', host='localhost', password='PWPWPW')

# server side cursor setup and execute
sscur = con.cursor(name='sentimentcursor')

QUERY = """
SELECT * FROM posts
LIMIT 60000000
"""
sscur.execute(QUERY)

cur = con.cursor()
i = 0
t1 = time.time()
with open('bulk_copy_dump.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar="'", quoting=csv.QUOTE_MINIMAL)
    for record in sscur:     # or cur2.fetchone, fetchmany...
        created_utc, subreddit, author, selftext, domain, id_val, title, num_comments, score, ups, downs, subredditid = record
        #print(created_utc)
        # compute sentiment using vader
        sentiment = analyzer.polarity_scores(selftext)

        # SQL INSERT
        if subredditid is None:
            subredditid = 'none'
        INSERT_QUERY = """INSERT INTO postssentiment (created_utc, subreddit, author, selftext, domain, id, title, num_comments, score, ups, downs, subreddit_id, vadersentiment) VALUES ('{:d}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{:d}', '{:d}', '{:d}', '{}', '{:.3f}');""".format(int(created_utc or 0), subreddit, author, str(selftext or '').replace('\n',' ').replace("'","''"), str(domain or '').replace('\n',' ').replace("'","''"), id_val, title.replace("'","''"), int(num_comments or 0), int(score or 0), int(ups or 0), int(downs or 0), str(subredditid or 0), sentiment['compound'])
        #print(INSERT_QUERY)
        cur.execute(INSERT_QUERY)


            
        i += 1
        if i % 20000 == 0:
            print('{} rows processed, {:.1f} rows per second.'.format(i, float(i)/(time.time()-t1+0.01)))
            #con.commit()
        pass

con.commit()
con.close()

20000 rows processed, 1235.5 rows per second.
40000 rows processed, 1360.2 rows per second.
60000 rows processed, 1490.4 rows per second.
80000 rows processed, 1527.0 rows per second.
100000 rows processed, 1534.1 rows per second.
120000 rows processed, 1437.1 rows per second.
140000 rows processed, 1438.5 rows per second.
160000 rows processed, 1418.5 rows per second.
180000 rows processed, 1377.1 rows per second.
200000 rows processed, 1380.5 rows per second.
220000 rows processed, 1408.8 rows per second.
240000 rows processed, 1412.9 rows per second.
260000 rows processed, 1412.6 rows per second.
280000 rows processed, 1389.3 rows per second.
300000 rows processed, 1413.0 rows per second.
320000 rows processed, 1488.9 rows per second.
340000 rows processed, 1562.0 rows per second.
360000 rows processed, 1634.0 rows per second.
380000 rows processed, 1650.3 rows per second.
400000 rows processed, 1622.4 rows per second.
420000 rows processed, 1617.3 rows per second.
440000 rows proce

In [22]:
# SERVER SIDE CURSOR FOR FETCHING POSTS FOR SENTIMENT ANALYSIS

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


#engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))

# connect:
con = None
con = psycopg2.connect(database = 'christle', user = 'christle', host='localhost', password='PWPWPW')

# server side cursor setup and execute
sscur = con.cursor(name='sentimentcursor')

QUERY = """
SELECT * FROM comments
LIMIT 25000000
"""
sscur.execute(QUERY)

cur = con.cursor()
i = 0
t1 = time.time()

for record in sscur:     # or cur2.fetchone, fetchmany...
    created_utc, subreddit, author, body, id_val, score, ups, downs, controversiality = record
    # compute sentiment using vader
    sentiment = analyzer.polarity_scores(body)
    try:
    # SQL INSERT 
        INSERT_QUERY = """INSERT INTO commentssentiment (created_utc, subreddit, author, body, id, score, ups, downs, controversiality, vadersentiment) VALUES ('{:d}', '{}', '{}', '{}', '{}', '{:d}', '{:d}', '{:d}', '{}', '{:.3f}');""".format(int(created_utc or 0), subreddit, author, body.replace("'","''"), id_val, int(score or 0), int(ups or 0), int(downs or 0), controversiality, sentiment['compound'])
        cur.execute(INSERT_QUERY)
    except:
        print(record)

    # WRITE FILE FOR BULK COPY (SPEED)
    #csv_row_list = [created_utc, subreddit, author, body.replace("'","''"), id_val, score, ups, downs, controversiality, sentiment['compound']]



    i += 1
    if i % 20000 == 0:
        print('{} rows processed, {:.1f} rows per second.'.format(i, float(i)/(time.time()-t1+0.01)))
    pass

con.commit()
con.close()

20000 rows processed, 1690.7 rows per second.
40000 rows processed, 1701.5 rows per second.
60000 rows processed, 1713.5 rows per second.
80000 rows processed, 1707.2 rows per second.
100000 rows processed, 1721.8 rows per second.
120000 rows processed, 1733.0 rows per second.
140000 rows processed, 1736.2 rows per second.
160000 rows processed, 1741.0 rows per second.
180000 rows processed, 1737.6 rows per second.
200000 rows processed, 1742.9 rows per second.
220000 rows processed, 1748.0 rows per second.
240000 rows processed, 1755.3 rows per second.
260000 rows processed, 1764.0 rows per second.
280000 rows processed, 1768.0 rows per second.
300000 rows processed, 1760.2 rows per second.
320000 rows processed, 1759.1 rows per second.
340000 rows processed, 1764.2 rows per second.
360000 rows processed, 1768.9 rows per second.
380000 rows processed, 1770.6 rows per second.
400000 rows processed, 1773.4 rows per second.
420000 rows processed, 1770.0 rows per second.
440000 rows proce

3460000 rows processed, 1840.4 rows per second.
3480000 rows processed, 1841.3 rows per second.
3500000 rows processed, 1842.1 rows per second.
3520000 rows processed, 1842.9 rows per second.
3540000 rows processed, 1843.7 rows per second.
3560000 rows processed, 1844.5 rows per second.
3580000 rows processed, 1845.3 rows per second.
3600000 rows processed, 1845.8 rows per second.
3620000 rows processed, 1846.5 rows per second.
3640000 rows processed, 1847.4 rows per second.
3660000 rows processed, 1848.0 rows per second.
3680000 rows processed, 1848.8 rows per second.
3700000 rows processed, 1849.5 rows per second.
3720000 rows processed, 1850.2 rows per second.
3740000 rows processed, 1851.0 rows per second.
3760000 rows processed, 1851.9 rows per second.
3780000 rows processed, 1852.6 rows per second.
3800000 rows processed, 1853.2 rows per second.
3820000 rows processed, 1853.8 rows per second.
3840000 rows processed, 1854.6 rows per second.
3860000 rows processed, 1855.3 rows per 

6880000 rows processed, 1743.8 rows per second.
6900000 rows processed, 1744.3 rows per second.
6920000 rows processed, 1744.0 rows per second.
6940000 rows processed, 1743.8 rows per second.
6960000 rows processed, 1744.0 rows per second.
6980000 rows processed, 1744.3 rows per second.
7000000 rows processed, 1744.8 rows per second.
7020000 rows processed, 1745.4 rows per second.
7040000 rows processed, 1745.9 rows per second.
7060000 rows processed, 1746.5 rows per second.
7080000 rows processed, 1747.0 rows per second.
7100000 rows processed, 1747.6 rows per second.
7120000 rows processed, 1748.1 rows per second.
7140000 rows processed, 1748.6 rows per second.
7160000 rows processed, 1749.2 rows per second.
7180000 rows processed, 1749.8 rows per second.
7200000 rows processed, 1750.3 rows per second.
7220000 rows processed, 1750.4 rows per second.
7240000 rows processed, 1750.1 rows per second.
7260000 rows processed, 1750.5 rows per second.
7280000 rows processed, 1751.0 rows per 

10300000 rows processed, 1799.5 rows per second.
10320000 rows processed, 1799.8 rows per second.
10340000 rows processed, 1800.1 rows per second.
10360000 rows processed, 1800.4 rows per second.
10380000 rows processed, 1800.4 rows per second.
10400000 rows processed, 1800.6 rows per second.
10420000 rows processed, 1801.0 rows per second.
10440000 rows processed, 1801.2 rows per second.
10460000 rows processed, 1801.5 rows per second.
10480000 rows processed, 1801.8 rows per second.
10500000 rows processed, 1802.1 rows per second.
10520000 rows processed, 1802.3 rows per second.
10540000 rows processed, 1802.4 rows per second.
10560000 rows processed, 1802.5 rows per second.
10580000 rows processed, 1802.8 rows per second.
10600000 rows processed, 1803.1 rows per second.
10620000 rows processed, 1803.3 rows per second.
10640000 rows processed, 1803.7 rows per second.
10660000 rows processed, 1803.9 rows per second.
10680000 rows processed, 1804.2 rows per second.
10700000 rows proces

13660000 rows processed, 1567.8 rows per second.
13680000 rows processed, 1568.3 rows per second.
13700000 rows processed, 1568.8 rows per second.
13720000 rows processed, 1569.3 rows per second.
13740000 rows processed, 1569.7 rows per second.
13760000 rows processed, 1570.1 rows per second.
13780000 rows processed, 1570.2 rows per second.
13800000 rows processed, 1570.6 rows per second.
13820000 rows processed, 1571.0 rows per second.
13840000 rows processed, 1571.5 rows per second.
13860000 rows processed, 1571.9 rows per second.
13880000 rows processed, 1572.3 rows per second.
13900000 rows processed, 1572.7 rows per second.
13920000 rows processed, 1573.3 rows per second.
13940000 rows processed, 1573.7 rows per second.
13960000 rows processed, 1574.2 rows per second.
13980000 rows processed, 1574.7 rows per second.
14000000 rows processed, 1575.2 rows per second.
14020000 rows processed, 1575.4 rows per second.
14040000 rows processed, 1575.8 rows per second.
14060000 rows proces

17020000 rows processed, 1583.6 rows per second.
17040000 rows processed, 1583.9 rows per second.
17060000 rows processed, 1584.2 rows per second.
17080000 rows processed, 1584.6 rows per second.
17100000 rows processed, 1584.9 rows per second.
17120000 rows processed, 1585.3 rows per second.
17140000 rows processed, 1585.7 rows per second.
17160000 rows processed, 1586.0 rows per second.
17180000 rows processed, 1586.4 rows per second.
17200000 rows processed, 1586.7 rows per second.
17220000 rows processed, 1587.1 rows per second.
17240000 rows processed, 1587.4 rows per second.
17260000 rows processed, 1587.8 rows per second.
17280000 rows processed, 1588.1 rows per second.
17300000 rows processed, 1588.5 rows per second.
17320000 rows processed, 1588.8 rows per second.
17340000 rows processed, 1589.2 rows per second.
17360000 rows processed, 1589.6 rows per second.
17380000 rows processed, 1589.8 rows per second.
17400000 rows processed, 1590.1 rows per second.
17420000 rows proces